# Module 28: LlamaIndex

**Data Framework for LLM Applications**

---

## 1. Objectives

- ✅ Understand LlamaIndex architecture
- ✅ Master data connectors and loaders
- ✅ Build different index types
- ✅ Implement query engines
- ✅ Create production RAG pipelines

## 2. Prerequisites

- [Module 21: RAG](../21_rag/21_rag.ipynb)
- [Module 26: LangChain](../26_langchain/26_langchain.ipynb)

## 3. LlamaIndex vs LangChain

### Key Differences

| Aspect | LlamaIndex | LangChain |
|--------|------------|----------|
| Focus | Data indexing & retrieval | Agent orchestration |
| Strength | RAG pipelines | General LLM apps |
| Philosophy | Data-centric | Chain-centric |
| Best For | Knowledge bases | Multi-step agents |

**Use Together**: LlamaIndex for retrieval + LangChain for orchestration!

### Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    LlamaIndex Pipeline                       │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  [Data Sources] ──→ [Loaders] ──→ [Documents]               │
│                                        ↓                     │
│                                  [Node Parser]               │
│                                        ↓                     │
│                                    [Nodes]                   │
│                                        ↓                     │
│                    [Embedding Model] + [Index]               │
│                                        ↓                     │
│                                  [Query Engine]              │
│                                        ↓                     │
│                                    [Response]                │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Install: pip install llama-index llama-index-llms-openai llama-index-embeddings-openai

# Set API key
# import os
# os.environ["OPENAI_API_KEY"] = "your-key"

from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    Document,
    Settings
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

print("LlamaIndex imported!")

## 4. Documents and Nodes

### Theory

| Concept | Description |
|---------|-------------|
| **Document** | Raw text with metadata |
| **Node** | Chunk of document with relationships |
| **Node Parser** | Splits documents into nodes |

In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

# Create documents manually
documents = [
    Document(
        text="LlamaIndex is a data framework for LLM applications. It provides tools for data ingestion, indexing, and querying.",
        metadata={"source": "intro", "category": "overview"}
    ),
    Document(
        text="Vector indexes are the most common type. They embed documents and enable semantic search. Other index types include list, tree, and keyword.",
        metadata={"source": "indexes", "category": "technical"}
    ),
    Document(
        text="Query engines combine retrieval with response synthesis. They can handle simple questions or complex multi-step queries.",
        metadata={"source": "query", "category": "technical"}
    )
]

print(f"Created {len(documents)} documents")
for doc in documents:
    print(f"  - {doc.metadata['source']}: {doc.text[:50]}...")

In [ ]:
# Parse documents into nodes
parser = SentenceSplitter(
    chunk_size=100,
    chunk_overlap=20
)

nodes = parser.get_nodes_from_documents(documents)

print(f"Split into {len(nodes)} nodes:")
for i, node in enumerate(nodes):
    print(f"  Node {i}: {node.text[:60]}...")

## 5. Data Loaders

### Built-in Loaders

| Loader | Source |
|--------|--------|
| SimpleDirectoryReader | Local files |
| PDFReader | PDF documents |
| WikipediaReader | Wikipedia articles |
| NotionReader | Notion pages |
| SlackReader | Slack messages |

In [ ]:
# Load from directory
# reader = SimpleDirectoryReader("./data")
# documents = reader.load_data()

# Load specific file types
# reader = SimpleDirectoryReader(
#     input_dir="./data",
#     required_exts=[".pdf", ".txt", ".md"]
# )

print("Directory reader example (uncomment to use)")

# Using LlamaHub for more loaders
# pip install llama-index-readers-web
# from llama_index.readers.web import SimpleWebPageReader
# loader = SimpleWebPageReader()
# documents = loader.load_data(["https://example.com"])

In [ ]:
# Custom loader example
from llama_index.core.readers.base import BaseReader
from typing import List

class CustomCSVReader(BaseReader):
    """Custom reader for CSV files."""
    
    def load_data(self, file_path: str) -> List[Document]:
        import csv
        
        documents = []
        with open(file_path, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                text = " | ".join(f"{k}: {v}" for k, v in row.items())
                documents.append(Document(
                    text=text,
                    metadata={"source": file_path}
                ))
        return documents

print("Custom CSV reader defined!")

## 6. Index Types

### Theory

| Index Type | How it Works | Best For |
|------------|-------------|----------|
| **VectorStoreIndex** | Embeds chunks, similarity search | Default choice |
| **SummaryIndex** | Traverses all nodes | Small datasets |
| **TreeIndex** | Hierarchical summarization | Q&A over docs |
| **KeywordTableIndex** | Keyword extraction | Keyword search |

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Configure settings
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Create vector index
index = VectorStoreIndex.from_documents(documents)

print("Vector index created!")

In [ ]:
# Summary Index - reads all nodes
from llama_index.core import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

print("Summary index created (good for small datasets)")

In [ ]:
# Persist index to disk
index.storage_context.persist(persist_dir="./storage")

# Load from disk later
from llama_index.core import StorageContext, load_index_from_storage

# storage_context = StorageContext.from_defaults(persist_dir="./storage")
# loaded_index = load_index_from_storage(storage_context)

print("Index persisted to ./storage")

## 7. Query Engines

### Theory

Query engines handle:
1. **Retrieval**: Find relevant nodes
2. **Postprocessing**: Rerank, filter
3. **Response Synthesis**: Generate answer

In [ ]:
# Basic query engine
query_engine = index.as_query_engine(
    similarity_top_k=2  # Retrieve top 2 chunks
)

# Query
response = query_engine.query("What is LlamaIndex?")

print(f"Answer: {response}")
print(f"\nSource nodes:")
for node in response.source_nodes:
    print(f"  - Score: {node.score:.3f}")
    print(f"    Text: {node.text[:80]}...")

In [ ]:
# Query with custom response mode
from llama_index.core.response_synthesizers import ResponseMode

# Response modes:
# - REFINE: Iteratively refine answer with each chunk
# - COMPACT: Compact chunks before synthesizing
# - TREE_SUMMARIZE: Build tree and summarize
# - SIMPLE_SUMMARIZE: Simple concatenation

query_engine = index.as_query_engine(
    response_mode=ResponseMode.COMPACT,
    similarity_top_k=3
)

response = query_engine.query("Explain different index types.")
print(f"Compact response: {response}")

## 8. Retrievers and Postprocessors

In [ ]:
# Custom retriever
retriever = index.as_retriever(
    similarity_top_k=5
)

# Retrieve nodes
nodes = retriever.retrieve("What are query engines?")

print(f"Retrieved {len(nodes)} nodes:")
for node in nodes:
    print(f"  - Score: {node.score:.3f} | {node.text[:60]}...")

In [ ]:
# Postprocessors for reranking and filtering
from llama_index.core.postprocessor import (
    SimilarityPostprocessor,
    KeywordNodePostprocessor
)

# Filter by minimum similarity
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.5)

# Apply to nodes
filtered_nodes = postprocessor.postprocess_nodes(nodes)
print(f"After filtering: {len(filtered_nodes)} nodes")

# Keyword filter
keyword_filter = KeywordNodePostprocessor(
    required_keywords=["query"],
    exclude_keywords=["tree"]
)

## 9. Chat Engine (Conversational)

In [ ]:
# Chat engine with memory
chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",  # Condenses history + retrieves context
    verbose=True
)

# Multi-turn conversation
response1 = chat_engine.chat("What is LlamaIndex?")
print(f"Bot: {response1}\n")

response2 = chat_engine.chat("What indexes does it support?")
print(f"Bot: {response2}\n")

response3 = chat_engine.chat("Which one should I use for semantic search?")
print(f"Bot: {response3}")

# Reset conversation
chat_engine.reset()

## 10. Advanced: Routers and Agents

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# Create multiple indexes
tech_docs = [d for d in documents if d.metadata.get("category") == "technical"]
overview_docs = [d for d in documents if d.metadata.get("category") == "overview"]

tech_index = VectorStoreIndex.from_documents(tech_docs)
overview_index = VectorStoreIndex.from_documents(overview_docs)

# Create tools
tech_tool = QueryEngineTool(
    query_engine=tech_index.as_query_engine(),
    metadata=ToolMetadata(
        name="technical_docs",
        description="Technical documentation about indexes and queries"
    )
)

overview_tool = QueryEngineTool(
    query_engine=overview_index.as_query_engine(),
    metadata=ToolMetadata(
        name="overview_docs",
        description="General overview and introduction"
    )
)

# Router query engine
router_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[tech_tool, overview_tool]
)

# Routes to appropriate index
response = router_engine.query("Give me an overview of LlamaIndex")
print(f"Routed response: {response}")

## 11. Production Tips

### Best Practices

| Aspect | Recommendation |
|--------|---------------|
| Chunking | 512-1024 tokens, 20% overlap |
| Embedding | text-embedding-3-small (cost) or large (quality) |
| Retrieval | Start with k=3-5, tune based on relevance |
| Reranking | Add cross-encoder reranker for quality |
| Caching | Cache embeddings and LLM responses |

In [ ]:
# Production configuration
from llama_index.core import Settings

# Optimize for production
Settings.chunk_size = 512
Settings.chunk_overlap = 50
Settings.num_output = 512  # Max response tokens

# Use caching
# from llama_index.core.storage.docstore import SimpleDocumentStore
# from llama_index.core.storage.index_store import SimpleIndexStore

print("Production settings configured!")

## 12. Interview Questions

**Q1: When should you use LlamaIndex vs LangChain?**
<details><summary>Answer</summary>

- **LlamaIndex**: Complex RAG, multi-document Q&A, structured data indexing
- **LangChain**: Agent workflows, tool use, multi-step reasoning
- **Together**: LlamaIndex retriever + LangChain agent
</details>

**Q2: What's the difference between Documents and Nodes?**
<details><summary>Answer</summary>

- **Document**: Raw source text with metadata (file, URL, etc.)
- **Node**: Chunk of document with:
  - Parent/child relationships
  - Embeddings
  - Source reference

Nodes enable better retrieval and context tracking.
</details>

**Q3: How would you improve RAG relevance?**
<details><summary>Answer</summary>

1. Better chunking (semantic, not fixed-size)
2. Hybrid search (dense + sparse)
3. Reranking with cross-encoder
4. Query expansion/rewriting
5. Metadata filtering
6. Parent document retrieval
</details>

## 13. Summary

| Component | Purpose |
|-----------|--------|
| Documents/Nodes | Data representation |
| Loaders | Ingest from sources |
| Indexes | Organize for retrieval |
| Query Engines | Retrieve + synthesize |
| Chat Engines | Conversational interface |

## 14. References

- [LlamaIndex Docs](https://docs.llamaindex.ai/)
- [LlamaHub (Data Loaders)](https://llamahub.ai/)
- [LlamaIndex GitHub](https://github.com/run-llama/llama_index)

---
**🎉 Congratulations! You've completed the full NLP + LLM Orchestration curriculum!**

Return to [Module 00: NLP Pipeline Overview](../00_nlp_pipeline/00_nlp_pipeline_overview.ipynb)